# IMDb Scraper

Workflow: 5 

Goal: Start with TMDB ids and get all data from IMDB.

Result: The file ```imdb_movie.csv``` is created.

In [1]:
import os
import time
import pandas as PD

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [3]:
# can't reload script in a different folder

import sys
sys.path.append('../models')
import imdb_scraper as MB 

## Scrape One Movie

In [4]:
scraper = MB.ImdbScraper()
scraper.open_imdb()

<selenium.webdriver.firefox.webdriver.WebDriver (session="2f54e946-12a0-b442-9ada-0576ed7bf678")>


In [5]:
title = 'Kaaliyan'
year = '2017'
# scraper.search_any(title)

In [6]:
# scraper.goto_movie_result(title, year)

In [7]:
# movie_dx = scraper.get_movie_data()
# movie_dx

In [8]:
movie_dx = scraper.get_movie_by_id('tt3170504')
movie_dx 

{'imdb_id': 'tt3170504',
 'title': 'Last Love Lost',
 'original_title': None,
 'year': '2015',
 'rating': 'Not Rated',
 'companies': None,
 'country': None,
 'language': None,
 'duration': 83,
 'directors': 'Jahmar Hill',
 'writers': 'Jahmar Hill',
 'actors': 'Jahmar Hill, Toni Belafonte, Talli Clemons',
 'genres': 'Drama',
 'synopsis': 'Antonio (Jahmar Hill) is awakened from a coma he was in for a year and a half after being shot by his father while trying to break up a domestic dispute between him and his mother. When ',
 'budget': None,
 'gross_us': None,
 'gross_worldwide': None,
 'score': '7.5',
 'votes': '15'}

In [9]:
scraper.close()

## Scrape Based on TMDB

In [10]:
INDEX_FILE = '../data/moviedb_movie.csv'
index_df = PD.read_csv(INDEX_FILE, dtype={'title': str, 'year': str})
index_df.head(3)
index_df.info()

,title,original_title,year,companies,country,language,run_time,crew,cast,poster,genres,collection,synopsis,budget,gross,score,votes,tmdb_id,imdb_id
0,1,1,2013,"Exclusive Media, Flat-Out Films",United States of America,English,112.0,"Paul Crowder, Mark Monroe, Michael Shevloff","Niki Lauda, Michael Schumacher, Lewis Hamilton",/4uIPXX8DjTsCzUAdtMKHTpojYLq.jpg,Documentary,NaN,Set in the golden era of Grand Prix Racing '1'...,NaN,NaN,7.4,59,217316,tt2518788
1,"10,000 BC","10,000 BC",2008,"Centropolis Entertainment, Legendary Entertain...",United States of America,English,109.0,"Roland Emmerich, Sarah Bradshaw, Tom Karnowski","Steven Strait, Camilla Belle, Cliff Curtis",/rnGR3EHkL4ryhQd50XBrtRrV8nq.jpg,"Adventure, Action, Drama, Fantasy",NaN,A prehistoric epic that follows a young mammot...,105000000.0,266000000.0,5.3,1766,7840,tt0443649
2,1000 Rupee Note,Ek Hazarachi Note,2016,NaN,India,Marathi (Marāṭhī),89.0,"Shrihari Sathe, Shrikant Bojewar",Sandeep Pathak,/pNNxwXAReV4kh7TCZGqBrl9I72v.jpg,Drama,NaN,Poor Parobudhi receives a thousand rupee note ...,NaN,NaN,6.9,7,318654,tt2937158


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17668 entries, 0 to 17667
Data columns (total 19 columns):
title             17668 non-null object
original_title    17668 non-null object
year              17668 non-null object
companies         11589 non-null object
country           13571 non-null object
language          17667 non-null object
run_time          16507 non-null float64
crew              16720 non-null object
cast              16002 non-null object
poster            17296 non-null object
genres            16109 non-null object
collection        1189 non-null object
synopsis          17537 non-null object
budget            2863 non-null float64
gross             1908 non-null float64
score             17668 non-null float64
votes             17668 non-null int64
tmdb_id           17668 non-null int64
imdb_id           17456 non-null object
dtypes: float64(4), int64(2), object(13)
memory usage: 2.6+ MB


In [11]:
# create jobs

index_1_df = index_df[0:5349]
index_2_df = index_df[5350:9999]
index_3_df = index_df[10000:13999]
index_4_df = index_df[14000:]

index_1_df.shape
index_2_df.shape
index_3_df.shape
index_4_df.shape

(5349, 19)

(4649, 19)

(3999, 19)

(3668, 19)

In [12]:
# investigate errors

error_titles = ['13 Times Evil', 'There Is a New World Somewhere', 'The Scarlet and the Black', 'The Sleeping Tiger', 
                'The Untold Story', 'Zombiegeddon', 'To Love The Soul Of A Woman', 'Tree Man']
index_df.loc[index_df['title'].isin(error_titles)][['title', 'year', 'imdb_id']]

,title,year,imdb_id
56,13 Times Evil,2016,NaN
15415,There Is a New World Somewhere,2015,tt3281502
15524,The Scarlet and the Black,1983,tt0086251
15634,The Sleeping Tiger,1954,tt0047505
15910,The Untold Story,1993,tt0103743
16288,To Love The Soul Of A Woman,2017,NaN
16452,Tree Man,2016,tt3246684
17642,Zombiegeddon,2003,tt0316946


In [13]:
#index_df.tail(20)['title']

In [14]:
scraper = MB.ImdbScraper()
scraper.open_imdb()
movie_ls = []
t0 = time.time()

for idx, row in index_3_df.iterrows():
    title = row['title']
    year = row['year']
    imdb_id = row['imdb_id']
    movie_dx = None

    try:
        movie_dx = scraper.get_movie_by_id(imdb_id)
    except Exception as ex:
        print('')
        print(f'Error 1: {title} {year} {imdb_id}')
        print(ex)

    if not movie_dx:
        try:
            scraper.open_imdb()
            scraper.search_any(title)
            found = scraper.goto_movie_result(title, year)
            if found:
                print(f'found by title: {title} {year} {imdb_id}')
                movie_dx = scraper.get_movie_data()
            else:
                print(f'NOT found by title: {title} {year}')
        except Exception as ex:
            print(f'Error 2: {title} {year} {imdb_id}')
            print(ex)

    if movie_dx:
        movie_ls.append(movie_dx)

scraper.close()
t1 = time.time()
print(f'movies: {len(movie_ls)}')
print(f'time: {(t1-t0)/60/60:.2f} hrs')

<selenium.webdriver.firefox.webdriver.WebDriver (session="3ca5e8be-f54d-9c4e-86be-5e5b474cff5a")>

Error 1: Nothing to Lose 2018 nan
Message: Unable to locate element: .title_wrapper

found by title: Nothing to Lose 2018 nan

Error 1: Once In A Lifetime Sessions with TLC 2018 nan
Message: Unable to locate element: .title_wrapper

NOT found by title: Once In A Lifetime Sessions with TLC 2018

Error 1: One Love 2011 nan
Message: Unable to locate element: .title_wrapper

found by title: One Love 2011 nan

Error 1: One More Shot 2018 nan
Message: Unable to locate element: .title_wrapper

found by title: One More Shot 2018 nan

Error 1: On the Streets of L.A. 1993 nan
Message: Unable to locate element: .title_wrapper

found by title: On the Streets of L.A. 1993 nan

Error 1: Our Earth - Our Oceans 3D 2015 nan
Message: Unable to locate element: .title_wrapper

Error 2: Our Earth - Our Oceans 3D 2015 nan
Message: Unable to locate element: //*[h3/text()="Titles"]


Error 1: Pac’s Scary Hallowe

NOT found by title: The 70% Club 2010

Error 1: The 9 Ball Diaries 2008 nan
Message: Unable to locate element: .title_wrapper

NOT found by title: The 9 Ball Diaries 2008

Error 1: The Amish: Shunned 2014 tt2547532
get_movie_data(): title error: tt2547532
NOT found by title: The Amish: Shunned 2014

Error 1: The Battle Of Chosin 2016 tt6186096
get_movie_data(): title error: tt6186096
NOT found by title: The Battle Of Chosin 2016

Error 1: The Biker Warrior Babe vs. The Zombie Babies From Hell 2014 nan
Message: Unable to locate element: .title_wrapper

found by title: The Biker Warrior Babe vs. The Zombie Babies From Hell 2014 nan

Error 1: The Bitcoin Experiment 2016 nan
Message: Unable to locate element: .title_wrapper

Error 2: The Bitcoin Experiment 2016 nan
Message: Unable to locate element: //*[h3/text()="Titles"]


Error 1: The Black Miami 2014 nan
Message: Unable to locate element: .title_wrapper

NOT found by title: The Black Miami 2014

Error 1: The Blackwell Ghost 2017 nan
Me

In [15]:
# projected time

#print(f'projected time: {(t1-t0)/60/60*18450/index_df.shape[0]:.2f} hrs')

In [16]:
movie_df = PD.DataFrame(movie_ls)
movie_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3956 entries, 0 to 3955
Data columns (total 19 columns):
imdb_id            3956 non-null object
title              3956 non-null object
original_title     540 non-null object
year               3956 non-null object
rating             2968 non-null object
companies          3656 non-null object
country            3938 non-null object
language           3899 non-null object
duration           3758 non-null float64
directors          3929 non-null object
writers            3625 non-null object
actors             3737 non-null object
genres             3953 non-null object
synopsis           3956 non-null object
budget             1210 non-null object
gross_us           771 non-null object
gross_worldwide    1235 non-null object
score              3921 non-null object
votes              3921 non-null object
dtypes: float64(1), object(18)
memory usage: 587.3+ KB


In [17]:
save_path = r'../data/imdb_mov_3.csv'
movie_df.to_csv(save_path, index=False)
f'{round(os.path.getsize(save_path) /1e6, 2)} mb'

'1.46 mb'

## Combine Job Files

In [18]:
#stop = 0 0
imdb_1_df = PD.read_csv('../data/imdb_mov_1.csv')
imdb_2_df = PD.read_csv('../data/imdb_mov_2.csv')
imdb_3_df = PD.read_csv('../data/imdb_mov_3.csv')
imdb_4_df = PD.read_csv('../data/imdb_mov_4.csv')

imdb_1_df.shape
imdb_2_df.shape
imdb_3_df.shape
imdb_4_df.shape

(5358, 19)

(4590, 19)

(3956, 19)

(317, 19)

In [19]:
movie_df = PD.concat([imdb_1_df, imdb_2_df, imdb_3_df, imdb_4_df])
movie_df.head(3)
movie_df.info()

,imdb_id,title,original_title,year,rating,companies,country,language,duration,directors,writers,actors,genres,synopsis,budget,gross_us,gross_worldwide,score,votes
0,tt2518788,1,NaN,2013,Not Rated,"Diamond Docs, Exclusive Media Group",USA,English,112.0,Paul Crowder,Mark Monroe,"Michael Fassbender, Niki Lauda, Lewis Hamilton","Documentary, History, Sport",Set in the golden era of Grand Prix Racing '1'...,NaN,NaN,NaN,8.0,3535.0
1,tt0443649,"10,000 BC",NaN,2008,PG-13,"Warner Bros., Legendary Entertainment",USA,English,109.0,Roland Emmerich,"Roland Emmerich, Harald Kloser","Camilla Belle, Steven Strait, Marco Khan","Action, Adventure, Drama","In the prehistoric past, D'Leh is a mammoth hu...",105000000,94784201.0,269784201,5.1,121431.0
2,tt2937158,Ek Hazarachi Note,NaN,2014,Not Rated,Infinitum Productions,India,Marathi,89.0,Shrihari Sathe,Shrikant Bojewar,"Devendra Gaikwad, Usha Naik, Pooja Nayak",Drama,"In this exploration of money and conscience, a...",NaN,2404.0,2404,7.3,309.0


<class 'pandas.core.frame.DataFrame'>
Int64Index: 14221 entries, 0 to 316
Data columns (total 19 columns):
imdb_id            14221 non-null object
title              14221 non-null object
original_title     1806 non-null object
year               14221 non-null int64
rating             10715 non-null object
companies          13133 non-null object
country            14151 non-null object
language           14032 non-null object
duration           13495 non-null float64
directors          14135 non-null object
writers            12959 non-null object
actors             13308 non-null object
genres             14211 non-null object
synopsis           14137 non-null object
budget             4180 non-null object
gross_us           2670 non-null float64
gross_worldwide    4315 non-null object
score              14121 non-null float64
votes              14121 non-null float64
dtypes: float64(4), int64(1), object(14)
memory usage: 2.2+ MB


In [20]:
# remove duplicates

movie_df = movie_df.drop_duplicates()
movie_df = movie_df.drop_duplicates(subset=['imdb_id'])
movie_df.shape

(13840, 19)

In [21]:
# fix up column types

movie_df['duration'] = movie_df['duration'].astype('Int64')
movie_df['votes'] = movie_df['votes'].astype('Int64')

In [22]:
save_path = r'../data/imdb_movie.csv'
movie_df.to_csv(save_path, index=False)
f'{round(os.path.getsize(save_path) /1e6, 2)} mb'

'5.06 mb'